In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

In [21]:
final_year = pd.read_csv("final_year.csv")

In [22]:
final_year.tail()

,ModuleCode,ModuleName,ModuleType,AssessmentName,AssessmentType,SubmissionType,CreditsWeight
19,CS3032,Mobile Multimedia,Computer Science,Assignment,CA,Programming,20
20,CS3032,Mobile Multimedia,Computer Science,Exam,Exam,Written,60
21,CS3033,Data Mining,Statistics,Assignment,CA,Programming,25
22,CS3033,Data Mining,Statistics,Assignment,CA,Programming,25
23,CS3033,Data Mining,Statistics,Exam,Exam,Written,50


In [29]:
cat_features = ["ModuleType","AssessmentType","SubmissionType"]
final_dummies = pd.get_dummies(final_year, columns=cat_features, drop_first=True)
final = final_dummies[['CreditsWeight','ModuleType_Psychology', 'ModuleType_Statistics', 'ModuleType_UX','AssessmentType_Exam', 'SubmissionType_Programming', 'SubmissionType_Quiz', 'SubmissionType_Written']].values
scaler = MinMaxScaler()
scaler.fit(final)
final = scaler.transform(final)

In [27]:
model = keras.models.load_model("predict_grades.h5")

In [56]:
predictions = model.predict(final)
predictions = pd.Series(test_predictions.reshape(24,))
prediction_df = pd.concat([final_year, predictions], axis=1)
prediction_df.rename(columns={prediction_df.columns[7] : "Model Predictions"} , inplace=True)
prediction_df.loc[prediction_df["Model Predictions"] > 100, "Model Predictions"] = 100

In [57]:
prediction_df

,ModuleCode,ModuleName,ModuleType,AssessmentName,AssessmentType,SubmissionType,CreditsWeight,Model Predictions
0,AP1033,Individual Differences,Psychology,Essay,CA,Written,100,71.036667
1,CS3031,Interaction Design,UX,Report,CA,Written,25,74.175964
2,CS3032,Interaction Design,UX,Report,CA,Written,25,74.175964
3,CS3033,Interaction Design,UX,Report,CA,Written,50,74.198990
4,AP3126,Health Psychology,Psychology,Quiz,CA,Quiz,10,77.671837
5,AP3126,Health Psychology,Psychology,Quiz,CA,Quiz,10,77.671837
6,AP3126,Health Psychology,Psychology,Critique,CA,Written,20,70.962967
7,AP3126,Health Psychology,Psychology,Behaviour Programme,NaN,Written,60,70.999817
8,CS3500,Software Engineering,Computer Science,Assignment,CA,Programming,30,94.185608
9,CS3500,Software Engineering,Computer Science,Lab Assignment,CA,Programming,30,94.185608


In [60]:
prediction_df[["ModuleName","Model Predictions"]].groupby("ModuleName").mean().sort_values("Model Predictions")

,Model Predictions
ModuleName,
Team Project,67.347397
Individual Differences,71.036667
Psychology of Aging,71.036667
Computing in Society,74.183640
Interaction Design,74.183640
Health Psychology,74.326614
Mobile Multimedia,79.064781
Software Engineering,79.064781
Data Mining,92.540070


In [65]:
round(np.average(prediction_df["Model Predictions"] ,weights=prediction_df["CreditsWeight"]),2)

72.81